In [1]:
import torch

In [2]:
import logging
from hydra import compose, initialize
from omegaconf import OmegaConf

from nn.autoeye.config import Config

if __name__ == "__main__":
    # global initialization
    initialize(version_base=None, config_path="nn/conf")
    cfg = compose(config_name="config")
    print(OmegaConf.to_yaml(cfg))
    log = logging.getLogger(__name__)
    Config.setup(cfg, log, train=True)

data:
  csv_files:
  - ./data/train.csv
  - ./data/test.csv
model:
  name: autoeye
  backbone: resnet
  version: 1.0
optim:
  beta1: 0.9
  beta2: 0.95
  weight_decay: 0.1
hyper:
  pretrained: true
  batch_size: 16
  dropout: 0.2
  lr: 0.003
  epochs: 10
  eval_iters: 1
  use_amp: false



In [ ]:
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dinov2_vits14

In [4]:
torch.save(dinov2_vits14, "./models/dino.pt")

In [ ]:
dino_model = torch.load("./models/dino.pt")
dino_model

In [3]:
from torch.cuda.amp import GradScaler
from torch.optim import AdamW

from nn.autoeye.model import AutoEye

# loading model
model = AutoEye().to(device=Config.device)
scaler = GradScaler(enabled=cfg.hyper.use_amp)
optimizer = AdamW(
    model.parameters(), lr=cfg.hyper.lr, betas=(cfg.optim.beta1, cfg.optim.beta2)
)

checkpoint = {
    "backbone": model.backbone.state_dict(),
    "classifier": model.classifier.state_dict(),
    "optimizer": optimizer.state_dict(),
    "scaler": scaler.state_dict(),
    "epochs": Config.get_trained_epochs() + 0,
    "accuracy": 0,
}


model.eval()
torch.save(checkpoint, Config.model_path)

/home/spectre/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from nn.autoeye.model import AutoEye


model = AutoEye()
checkpoint = torch.load("./models/autoeye_resnet.pt")
model.load(checkpoint)

resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

model.backbone = resnet

checkpoint = {
    "backbone": model.backbone.state_dict(),
    "classifier": model.classifier.state_dict(),
    "optimizer": checkpoint["optimizer"],
    "scaler": checkpoint["scaler"],
    "epochs": 0,
    "accuracy": 0,
}
torch.save(checkpoint, "./models/autoeye_resnet.pt")

Using cache found in /home/spectre/.cache/torch/hub/pytorch_vision_v0.10.0
/home/spectre/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/spectre/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
